In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import math
import csv

#選択行削除
import re
from xml.sax.saxutils import unescape

In [3]:
def normalize_XBRL(XBRL):
    with open(XBRL, encoding='utf-8') as f:
        html = f.read()
    # 「（」「）」はすべて半角に
    html = html.replace("（", "(")
    html = html.replace("）", ")")
    #tableタグだけ残す，それ以外のタグで囲まれていない部分を削除
    html  = re.sub("<table", "table\n<", html)
    html  = re.sub("</table", "tableend\n<", html)
    html  = re.sub("<br />", "", html)
    html  = re.sub("<td", "td<", html)
    html  = re.sub("</td>", "tdend\n", html)
    html  = re.sub("</tr>", "trend", html)
    
    text=""
    for i in html:
        text=text+i
        if i ==">":
            text=re.sub("<.*>", "", text)
            text=re.sub("{.*}", "", text)
            if text[-1:]!="\n":
                text = text+"\n"
    text1=text

    #テキスト処理
    text1=re.sub("&#160;", "None", text1)
    text1=re.sub("△\n", "-", text1)
    text1=re.sub("△", "-", text1)
    text1=re.sub(".root", "-", text1)
    text1=re.sub(",", "", text1)
    text1=re.sub("．", ".", text1)
    text1=re.sub("\ufeff\n", "", text1)
    text1=re.sub("－", "None", text1)
    text1=re.sub("：", "", text1)
    text1=re.sub(" ", "", text1)
    text1=re.sub("\n\n", "\n", text1)
    text1=re.sub("　", "", text1)
    text1=re.sub("０", "0", text1)
    text1=re.sub("１", "1", text1)
    text1=re.sub("２", "2", text1)
    text1=re.sub("３", "3", text1)
    text1=re.sub("４", "4", text1)
    text1=re.sub("５", "5", text1)
    text1=re.sub("６", "6", text1)
    text1=re.sub("７", "7", text1)
    text1=re.sub("８", "8", text1)
    text1=re.sub("９", "9", text1)
    

    return(text1)

In [4]:
def text_tag(text):
    #予測がいつまでなのかを取得(基本的に　３．　OO年度連結業績予測などになっているため，それを元に取得)
    pre_cnt=0
    pre = ""
    for i in range(len(text),-1,-1):
        if "連結業績予想(" in text[i-7:i]:
            pre_cnt=i
            break
    for i in range(pre_cnt,-1,-1):
        if "." in text[i-1:i]:
            pre = text[i:pre_cnt-1]
            break

    #tdで囲まれた部分の改行は削除して，tdを削除
    tx=""
    text1=""
    td_flag=False
    for i in text:
        #print(i)
        if "tdend" in text1:
            td_flag=False
        if "td" in text1:
            td_flag=True

        if td_flag:
            if i=="\n":
                i=""
        text1+=i
    text1=re.sub("tdend", "", text1)
    text1=re.sub("td", "\n", text1)
    text1=re.sub(r"\n+", "\n", text1)
    return(text1,pre)

In [5]:
def create_text_list(text):

    #tableタグで囲まれた部分をリストに入れる
    text_list = []
    table_cnt_first=0
    table_cnt_end=0
    non_text_list = []
    non_table_cnt_first=0
    append_text=""
    non_text=""
    for i in range(1,len(text)):
        if text[i-8:i] == "tableend":
        #tableタグに囲まれた文章をlistに追加
            table_cnt_end = i-8
            append_text=text[table_cnt_first:table_cnt_end]+"\n"
            append_text=re.sub("trend\ntrend","trend\n",append_text)
            append_text=re.sub("tableendtable","",append_text)
            append_text=re.sub(r'\n+', "\n",append_text)
            text_list.append(append_text)
            table_cnt_first=0
            table_cnt_end=0
            #tableタグに囲まれている部分を追加したので囲まれていない部分のカウント初期化
            non_table_cnt_first=i

    #tableタグの始まりの判定    
        elif text[i-5:i] == "table":
            if table_cnt_first==0:
                table_cnt_first=i

            #tableタグに囲まれていない部分をlistに追加
            non_text=text[non_table_cnt_first:i-5]
            non_text_list.append(non_text)
    
    #listの中に（注）がある部分は削除する         
    pop_list=[]
    cnt=0
    for i in text_list:

        #if "(注)" in i :
          #  pop_list.append(cnt)
        if "(参考)" in i:
            pop_list.append(cnt)
        elif "(役職名)" in i:
            pop_list.append(cnt)
        cnt +=1

    for i in pop_list[::-1]:
        #print(i)
        text_list.pop(i)
    return(text_list,non_text_list)

In [6]:
def create_table_list(text_list,column_dic):
    new_li=[]
    n_cnt=0
    col=""
    column_li=[i for i in column_dic.keys()]
    #一番可能性が高い表をlistのトップに持ってくる
    for cnt,table in enumerate(text_list):
        n_cnt=0
        col=""
        for i,word in enumerate(table,1):
            if word=="\n":
                #columnになる可能性のある単語がcolumn辞書に存在する場合は高い確率で目的の表であると考えられる
                col = table[n_cnt:i-1]
                col= re.sub("trend","",col)
                if col in column_li:
                    new_li.append(table)
                    break
                n_cnt=i
    return(new_li)

In [7]:
def create_list(table_list):
    number_list = []
    col_list=[]
    number = []
    col=""
    num_cnt=0
    col_cnt=0
    len_col=0
    add_col=0
    non_cnt=0
    for table in table_list:
        if "table" in table:
            table=table[table.find("table")+5:]
        for i in range(0,len(table)):
            if table[i:i+5] !="trend":
                if table[i:i+1] == "\n":
                    col=table[col_cnt+1:i]
                    add_col+=1
                    if col=="" and col_list!=[] and len_col==0:
                        len_col=len(col_list)
                    if col!="":
                        col_list.append(col)
                    col_cnt=i
                
            else:
                col=table[col_cnt+1:i]
                col_list.append(col)
                #追加したコラムが１つだけなら削除してやり直し
                if add_col==1:
                    add_col=0
                    col_list.pop(-1)
                    col_cnt=i+5
                    continue
                add_col=0
                num_cnt=i+5
                break
                
        #col_listの先頭がNoneでなければNoneを追加
        if (col_list!=[]) and col_list[non_cnt]!="None":
            #col_list.insert(len(column),"None")
            col_list.insert(non_cnt,"None")
            non_cnt=len(col_list)
            
        #column以降にある数字の部分を回す
        for i in range(num_cnt,len(table)+1):
            if table[i:i+1] == "\n":
                if table[i-5:i]=="trend":
                    if number!=[]:
                        number.append(table[num_cnt+1:i-5])
                        number_list.append(number)
                        number=[]
                        num_cnt=i

                elif table[i-5:i]!="trend":
                    number.append(table[num_cnt+1:i])
                    num_cnt=i
                    
    #[col_list.insert(k, col_list[k]) for k in range(len_col-1,-1,-1)]
    
    return(col_list,number_list)

In [8]:
def create_NoneAndCol(column,number):
    #columnをNoneを起点に分ける
    number_list=[]
    column_list=[]
    pop_list=[]
    cn=1
    for i in range(1,len(column)):
        if column[i] =="None":
            column_list.append(column[cn:i])
            cn=i+1
        elif i==len(column)-1:
            column_list.append(column[cn:i+1])
            
            
    #numberにNoneを入れる
    for cnt,num in enumerate(number):
        #if len(num) <3:
          #  pop_list.append(cnt)
        if ("円銭" in num) or ("百万円" in num):
            if "None" not in num:
                num.insert(0,"None")
        
        #後々Noneがあるとエラーになるのでいらないものは削除
        if num[0] != "None":
            num[0] = re.sub("None","",num[0])
        number_list.append(num)
        

    for i in pop_list[::-1]:
        #print(i)
        number_list.pop(i)
    return(column_list,number_list)

In [9]:
def create_index(number):
    index_list=[[]*i for i in range(0,len(number))]
    in_cnt=0
    index_list[in_cnt].append("None")

    for i in range(1,len(number)):
        if number[i][0]== "None":
            in_cnt+=1
            index_list[in_cnt].append("None")
        else:
            index_list[in_cnt].append(number[i][0])
    return(index_list)

In [10]:
def create_new_number_list(number,column_list):
    new_num=[]
    cn=0
    for cnt,num in enumerate(number):
        if num[0]== "None":
            if len(new_num)!=len(column_list):
                if len(column_list[cn])>2:

                    for i in range(0,len(num),2):
                        if i>len(column_list[cn])-1:
                            continue
                        if (len(num)-1) > (len(column_list[cn])):
                            column_list[cn].insert(i,column_list[cn][i])

                        else:
                            break
                new_num.append([column_list[cn][k]+num[k+1] for k in range(0,len(column_list[cn]))])
                new_num[cn].insert(0,"None")
                number[cnt]=new_num[cn]
                if cn!=len(column_list)-1:
                    cn+=1
    
    return(number)

In [65]:
def create_DataFrame(number,index_list):
    #最大でも４つまでの表しか作成しない．目的の表は２つだけなので
    new_df1 = pd.DataFrame

    cn=1
    num_cnt=0
    for cnt,index in enumerate(index_list):
        if index!=[] and cn==1:
            new_df1=pd.DataFrame([number[k][1:] for k in range(num_cnt,len(index)+num_cnt)],index=index)
            num_cnt+=len(index)
            cn+=1
    return(new_df1)

In [12]:
def reset_column(df1,df2):
    sh1=df1.shape
    sh2 = df2.shape
    #column1をリセット
    df1.columns = range(sh1[1])
    #column2をリセット
    df2.columns = range(sh2[1])
    
    return(df1,df2)

In [13]:
def reset_column1(df1):
    sh1=df1.shape

    #column1をリセット
    df1.columns = range(sh1[1])

    return(df1)

In [14]:
def select_df(df2,df3,df4,pre_term):
    
    try:
        if ("通期" in str(df2.index)) or( "予想" in str(df2.index ))or("累計" in str(df2.index)):
            return(df2)
        elif ("通期" in str(df3.index)) or( "予想" in str(df3.index ))or("累計" in str(df3.index)): 
            return(df3)
        elif ("通期" in str(df4.index)) or( "予想" in str(df4.index ))or("累計" in str(df4.index)):
            return(df4)
        
        else:
            index_list=[]
            index_list = [i for i in df2.index]
            for j in index_list:
                if pre_term[:4] in j:
                    return(df2)
                pass
            
            index_list = [i for i in df3.index]
            for j in index_list:
                if pre_term[:4] in j:
                    return(df3)
                pass
            
            index_list = [i for i in df4.index]
            for j in index_list:
                if pre_term[:4] in j:
                    return(df4)
                pass
    except:
        pass

In [15]:
def create_df1_column(df1,column_list):
    df1_col=[ i for i in df1.columns]
    cnt_0 = df1_col.count(0)
    new_col=[]
    cnt=1
    end=0
    try:
        for zero_cnt in range(0,cnt_0):
            for i in range(cnt,len(df1_col)):
                if df1_col[i]==0:
                    #columnとdfの長さが同じ時（表１）
                    if len(column_list[zero_cnt]) == i:
                        new_col= column_list[0]
                    #columnとdfの長さが同じ時（表２）
                    elif (zero_cnt!=0) and (len(column_list[zero_cnt]) == i-end+1):
                        new_col= column_list[0]
                    else:
                        cnt=i+1
                        break
                #エラーが起きないように最後になったらforを抜ける        
                elif i==len(df1_col)-1:
                    cnt=i+3
                    break
            #抜けた後，column数が同じならそのまま加える
            if (len(column_list[zero_cnt]) == i-end+1):
                for k in column_list[zero_cnt]:
                    new_col.append(k)
            else:
                #0の手前までのcolumnを作成
                cn=0
                for j in range(end,cnt-1):
                    if j %2==0:
                        new_col.append(column_list[zero_cnt][cn])
                    else:
                        new_col.append(column_list[zero_cnt][cn])
                        cn+=1
                end=j+2
        df1.loc["index"]=new_col
    except:
        df1.loc["index"]=new_col
    return(df1,zero_cnt+1)


In [109]:
def create_df_column(df1,column_list):
    try:
        df1_col=[ i for i in df1.columns]
        new_col=[]
        cn=0
        for j in range(0,len(df1_col)):
            if j %2==0:
                new_col.append(column_list[0][cn])
            else:
                new_col.append(column_list[0][cn])
                cn+=1
        df1.loc["index"]=new_col
        return(df1)
    except:
        return(df1)

In [17]:
def check_text(text_list,non_text):
    new_text_list=[]
    new_text=""

    for txt in text_list:
        if txt.count("。")>=1:
            new_text+=txt
    new_text_list.append(new_text)
    
    new_text=""
    for txt in non_text:
        if txt.count("。")>=1:
            new_text+=txt
    new_text_list.append(new_text)
    return(new_text_list)

In [18]:
def NoPredict_or(NoPredict_text):
    cnt=0
    cnt=0
    for i in NoPredict_text:
        if ("未定"in i)or ("控え" in i) or ("困難" in i)or("M&A" in i) or("コロナウイルス" in i) or("合理的に算出" in i):
            cnt+=1
    if cnt>0:
        return(True)
    else:
        return(False)

In [59]:
col_dic={
    "総資産":"TotalAssets",
    "純資産":"NetAssets",
    "自己資本比率":"CapitalAdequacyRatio"
}

In [20]:
company = pd.read_csv("TDNet_full_data/new_company_list.csv",index_col=0)

In [25]:
df = pd.read_csv("new2/df_sum/this/{}.csv".format(company.index[i]),encoding="cp932")

In [28]:
df["number"][0]

20170619410034

In [44]:
path = "xbrl_data/{}/{}.htm".format(company.index[i],df["number"][j])

In [45]:
path

'xbrl_data/13010/20170905468428.htm'

In [102]:
text=normalize_XBRL(path)
text,pre_term = text_tag(text)
text_list,non_text=create_text_list(text)
NoPredict_text=check_text(text_list,non_text)
table_list=create_table_list(text_list,col_dic)
column,number=create_list(table_list)
column_list,number = create_NoneAndCol(column,number)
index_list=create_index(number)
df=create_DataFrame(number,index_list)
#df1,zero_cnt,=create_df1_column(df1,column_list)

In [53]:
import os
def create_dic(dic):
    #フォルダを作成する
    try:
        os.makedirs(dic)
    except FileExistsError:
        pass

In [69]:
import tqdm

In [82]:
out_list=[]
for i in tqdm.tqdm(range(0,len(company.index))):
    if not company.index[i] in non_com:
        df = pd.read_csv("new2/df_sum/this/{}.csv".format(company.index[i]),encoding="cp932")

        for j in range(0,len(df)):

            path = "xbrl_data/{}/{}.htm".format(company.index[i],df["number"][j])
            out_csv='new2/xbrl_asset/{}/{}.csv'.format(company.index[i],df["number"][j])
            dic = 'new2/xbrl_asset/{}'.format(company.index[i])
            try:
                text=normalize_XBRL(path)
                text,pre_term = text_tag(text)
                text_list,non_text=create_text_list(text)
                NoPredict_text=check_text(text_list,non_text)
                table_list=create_table_list(text_list,col_dic)
                column,number=create_list(table_list)
                column_list,number = create_NoneAndCol(column,number)
                index_list=create_index(number)
                df1=create_DataFrame(number,index_list)
                df1,zero_cnt,=create_df1_column(df1,column_list)

                #フォルダを作成する
                create_dic(dic)
                df1.to_csv(out_csv)

            except:
                out_list.append(path)

100%|███████████████████████████████████████| 2972/2972 [25:14<00:00,  1.96it/s]


In [71]:
df1

,0,1,2,3
None,百万円,百万円,％,円銭
2022年3月期,130460,42174,32.7,3969.73
2021年3月期,116331,39975,34.7,3753.90
index,総資産,純資産,自己資本比率,1株当たり純資産


In [83]:
out_list

['xbrl_data/17240/20200213463368.htm',
 'xbrl_data/17240/20210210461632.htm',
 'xbrl_data/17240/20220214588928.htm',
 'xbrl_data/22010/20190513424390.htm',
 'xbrl_data/24890/20201105416794.htm',
 'xbrl_data/34070/20190510420533.htm',
 'xbrl_data/63610/20220512542868.htm',
 'xbrl_data/65390/20210512412922.htm',
 'xbrl_data/65390/20220511541079.htm',
 'xbrl_data/88500/20211103423827.htm',
 'xbrl_data/90310/20190206471170.htm',
 'xbrl_data/93250/20191108422492.htm',
 'xbrl_data/93250/20201113423946.htm',
 'xbrl_data/93250/20211112433880.htm']

In [75]:
company.loc[29820]

name                            株式会社ＡＤワークスグループ
class                                     不動産業
SettlementDate                          12月31日
end               2022年12月期第１四半期決算短信〔日本基準〕(連結)
start             2020年12月期第１四半期決算短信〔日本基準〕(連結)
Name: 29820, dtype: object

# out_listの処理

In [124]:
for i in range(0,len(out_list)):
    try:
        out_csv='new2/xbrl_asset/{}/{}.csv'.format(out_list[i][10:15],out_list[i][16:-4])
        dic = 'new2/xbrl_asset/{}'.format(out_list[i][10:15])
        text=normalize_XBRL(out_list[i])
        text,pre_term = text_tag(text)
        text_list,non_text=create_text_list(text)
        NoPredict_text=check_text(text_list,non_text)
        table_list=create_table_list(text_list,col_dic)
        column,number=create_list(table_list)
        column_list,number = create_NoneAndCol(column,number)
        index_list=create_index(number)
        df1=create_DataFrame(number,index_list)
        df1=create_df_column(df1,column_list)
        
        #フォルダを作成する
        create_dic(dic)
        df1.to_csv(out_csv)
    except:
        print(out_list[i])

xbrl_data/24890/20201105416794.htm
xbrl_data/63610/20220512542868.htm


# 例外処理

In [143]:
path="xbrl_data/24890/20201105416794.htm"
#path="xbrl_data/63610/20220512542868.htm"

In [144]:
text=normalize_XBRL(path)
text,pre_term = text_tag(text)
text_list,non_text=create_text_list(text)
NoPredict_text=check_text(text_list,non_text)
table_list=create_table_list(text_list,col_dic)
column,number=create_list(table_list)
column_list,number = create_NoneAndCol(column,number)
index_list=create_index(number)
df1=create_DataFrame(number,index_list)
#df1=create_df_column(df1,column_list)

IndexError: list index out of range

In [147]:
n=["百万円","百万円","％","円銭"]
n1=[20474,12991,62.4,307.42]
n2=[18986,12448,64.6,294.90]
column=["総資産","純資産","自己資本比率","1株当たり純資産"]

In [148]:
df1=pd.DataFrame([n,n1,n2,column],index=["None","2021年3月期第2四半期","2020年3月期","column"])

In [149]:
df1

,0,1,2,3
None,百万円,百万円,％,円銭
2021年3月期第2四半期,20474,12991,62.4,307.42
2020年3月期,18986,12448,64.6,294.9
column,総資産,純資産,自己資本比率,1株当たり純資産


In [150]:
out_csv='new2/xbrl_asset/24890/20201105416794.csv'
df1.to_csv(out_csv)

In [77]:
non_com=[]
for i in tqdm.tqdm(range(0,len(company.index))):
    try:
        dic = 'new2/xbrl_asset/{}'.format(company.index[i])
        df = pd.read_csv("new2/df_sum/this/{}.csv".format(company.index[i]),encoding="cp932")
        #フォルダを作成する
        create_dic(dic)
    except:
        non_com.append(company.index[i])

100%|█████████████████████████████████████| 2972/2972 [00:01<00:00, 1514.82it/s]


In [79]:
company.loc[non_com[1]]

name                                    黒谷株式会社
class                                      卸売業
SettlementDate                            8月末日
end               2022年８月期　第２四半期決算短信〔日本基準〕（連結）
start             平成29年８月期　第３四半期決算短信〔日本基準〕（連結）
Name: 31680, dtype: object

In [80]:
non_com

[29820,
 31680,
 32710,
 35530,
 39990,
 47220,
 49670,
 56030,
 65880,
 67860,
 73390,
 74850,
 83030,
 94680]